<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/3.0.0_beta1/OpenHosta/tree/3.0.0_beta1/docs/SmokeTest_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instal OpenHosta

In [1]:
!pip install --upgrade -qqq uv

# If you need to test a pre release uncomment:
VERSION="@3.0.0_beta1"

!uv pip install -U \
    "git+https://github.com/hand-e-fr/OpenHosta.git$VERSION"

Using Python 3.12.3 environment at: /home/ebatt/VSCode_GitRepos/.venv
Resolved 9 packages in 151ms                                         
Audited 9 packages in 0.48ms


In [2]:
import OpenHosta; print(OpenHosta.__version__)

3.0.0


# Install Ollama

This is to run a local model and have zero dependancies to externa API provides.

In [ ]:
# This seems not to be accepted by google colab anymore. 
# Run it in a separtated terminal i refused
#!apt install -y screen
#!curl -fsSL https://ollama.com/install.sh | sh
#!screen -dmS ollama ollama serve

# So we just show how to install Ollama on linux
!curl -fsSL https://ollama.com/install.sh | sh

We test is ollama is available and how fast Qwen3 runs on it.

In [3]:
!ollama run qwen3:4b hello --verbose  2>&1 | grep -E ":"

total duration:       2.679660793s
load duration:        2.002796383s
prompt eval count:    9 token(s)
prompt eval duration: 145.13192ms
prompt eval rate:     62.01 tokens/s
eval count:           116 token(s)
eval duration:        529.879618ms
eval rate:            218.92 tokens/s


# Use OpenHosta

In [1]:

from OpenHosta import config

# You can replace with your own API (OpenAI chat/completion compatble)
config.DefaultModel.base_url = "http://localhost:11434/v1"
config.DefaultModel.model_name = "qwen3:4b"
config.DefaultModel.api_key = "not used by ollama local api"


**We test that the API is working with a simple call**

`ask()` makes a very simple call to the API without adding any hidden prompt.


In [2]:
from OpenHosta import ask

ask("Hello World!")

'<think>\nOkay, the user said "Hello World!" and I need to respond. Let me think about how to approach this.\n\nFirst, "Hello World!" is a common greeting, so I should acknowledge it. Maybe start with a friendly reply. But what\'s the best way to respond? Should I just say hello back, or add something else?\n\nWait, the user might be testing if I can handle simple interactions. Maybe I should respond in a way that shows I\'m active and ready to help. Let me make sure the response is polite and open-ended. \n\nI should also check if there\'s any specific context I\'m missing. But since the user just said "Hello World!", it\'s probably just a greeting. So a simple "Hello!" back should work. But maybe add a bit more to make it more engaging. For example, ask how they\'re doing or offer assistance. \n\nHmm, but the user might not want a long response. Maybe keep it short and friendly. "Hello! How can I assist you today?" That sounds good. It\'s concise and invites them to ask for help. \n\

Most OpenHosta functions are avaiale in sync and async flavor.

Async versions are provided by adding `_async` or by importing from  `from OpenHosta.asynchrone`


In [ ]:
from OpenHosta.asynchrone import ask

# Another way to get the same result is:
# from OpenHosta import ask_async as ask

await ask("Hello World!")


'<think>\nOkay, the user said "Hello World!" so I need to respond appropriately. Since they greeted me, I should return a friendly greeting. Maybe say "Hello!" back and offer help. Keep it simple and welcoming. Let them know I\'m here to assist with anything they need. No need for any other text, just a basic response.\n</think>\n\nHello! How can I assist you today? 😊'

# The emulate function

The main purpose of OpenHosta is to let you organize your code in python and let you decide later if you need to implement function bodies in native python or let an LLM do the job. 

This is really pythonic as types and doc strings will remain unchanged if you decide to replace `return emulate()` by your own code.

In [9]:
from OpenHosta import emulate

def translate(text:str, language:str)->str:
    """
    This function translates the text in the “text” parameter into the language specified in the “language” parameter.
    """
    return emulate()

result = translate("Hello World!", "French")
print(result)

Bonjour le monde!


In very simple words, `emulate()` does the inspection of the function it is called from and buld a prompt that delegated computation of the function to an LLM.
This is very usefull for NLP based functions, but also **smart** decisions in your workflow.

In [60]:
from enum import Enum

class Choice(Enum):
    """
    WebSearch = "Generic Subject, but not enough confidence in LLM knowledge"
    UseLLM    = "Generic Subject, obviouse answer."
    UserRAG   = "Company private subject, need to search for internal documentation through RAG"
    """
    WebSearch = "WebSearch"
    UseLLM    = "UseLLM"
    UserRAG   = "UserRAG"

In [61]:
from OpenHosta import emulate

def decide_if_search_or_answer(query: str) -> Choice:
    """
    Decide to search for answer elements in one of the multiple Choice locations.
    
    When the LLM is usure about the correct answer it shall query external data sources.
    
    Args:
        subject (str): The question that we are working on
    Return:
        Choice: the best strategy to use
    """
    return emulate()

In [62]:
decide_if_search_or_answer("april 2021 is after mars 2020?")


<Choice.UseLLM: 'UseLLM'>

In [45]:
decide_if_search_or_answer("who won the last NFL cup?")


<Choice.WebSearch: 'WebSearch'>

In [65]:
decide_if_search_or_answer("when was the first soccer world cup?")


<Choice.UseLLM: 'UseLLM'>

Ok, but can I see the prompt behind and what was really sent to the API?
-YES-

And the thinking and answer of the LLM ?
-YES-

In [66]:
from OpenHosta import print_last_prompt

print_last_prompt(decide_if_search_or_answer)

System prompt:
-----------------
You will act as a simulator for functions that cannot be implemented in actual code.

I'll provide you with function definitions described in Python syntax. 
These functions will have no body and may even be impossible to implement in real code, 
so do not attempt to generate the implementation.

Instead, imagine a realistic or reasonable output that matches the function description.
I'll ask questions by directly writing out function calls as one would call them in Python.
Respond with an appropriate return value, without adding any extra comments or explanations.
If the provided information isn't enough to determine a clear answer, respond simply with "None".
If assumptions need to be made, ensure they stay realistic, align with the provided description.

Here's the function definition:

```python
# Python enum Choice definition.
# When you return a Choice, print the enum member value as a string. I will identify the corresponding enum member.
class C

# Build a workflow agent

Let's see how to impement an agent with memory within an object

In [ ]:
import asyncio
from dataclasses import dataclass
from OpenHosta.asynchrone import emulate, closure

from enum import Enum

class SubjectsInMemoryStore(Enum):
    FamilyMemberLinks = "FamilyMemberLinks"
    PeopleNames = "PeopleNames"
    Grocery = "Grocery"
    CalendarEvents = "CalendarEvents"
    NotValidSubject = "NotValidSubject"
    
class ActionType(Enum):
    INSERT="INSERT"
    UPDATE="UPDATE"
    DELETE="DELETE"
    SELECT="SELECT"
    
    
@dataclass
class UserDetails:
    FirstName: str
    LastName: str
    Age: int

class MyAgentWithMemory:
    """
    This agent records elements from the conversation only if they are related to specific subkects
    """
    
    CurrentUserId:str = None
    MemoryElements:dict = {}
    
    ChatLogs = []
    
    async def is_about_element_that_we_record(self, sentence: str) -> SubjectsInMemoryStore:
        """
        This function takes a sentenc and identify if there is element to record.
        
        We only record elements on subjects that we are allowed to.

        Args:
            sentence (str): a snippet of text from a conversation

        Returns:
            SubjectsInMemoryStore: In what category to store if we store something
        """
        # UserName variable and value will be worwarded to the LLM
        SpeakerName=self.MemoryElements.get(SubjectsInMemoryStore.PeopleNames, {}).get(self.CurrentUserId)
        return await emulate()
    
    async def detect_action_type(self, sentence:str) -> ActionType:
        """
        Identify the type of action that is implicitly requested in the sentence.
        
        The sentence is produced by us user of our system. Our system has a memory.
        What action should be do on our memory to best answer user expectations?
        
        Return:
            ActionType
        """
        return await emulate() 
        
    async def is_telling_who_he_is(self, sentence:str) -> bool:
        """
        The speaker is telling about who he is in this sentence.

        Args:
            sentence (str): a sentence made by a speaker

        Returns:
            bool: True if we learn about his name or forstname
        """
        return await emulate()
    
    async def fill_user_details(self, sentence:str, previouse_details:UserDetails)->UserDetails:
        """
        Identifies data fields from the sentence

        Args:
            sentence (_type_): what the user say
            previouse_details: What we new about this user
            
        Return:
            Filled UserDetails object for this user
        """
        return await emulate()
    
    
    async def answer_to(self, sentence:str)->str:
        """
        Proces the use input, handle memory, than answer.

        Args:
            sentence (str): what the client say

        Returns:
            str: what the agent answers
        """
        if self.CurrentUserId is None:
            tells_who_he_is = await self.is_telling_who_he_is(sentence)
            print(tells_who_he_is)
            if tells_who_he_is:
                who = await self.fill_user_details(sentence, None)
                self.CurrentUserId = "you"
                if SubjectsInMemoryStore.PeopleNames not in self.MemoryElements:
                    self.MemoryElements[SubjectsInMemoryStore.PeopleNames] = {}
                self.MemoryElements.get(SubjectsInMemoryStore.PeopleNames)[self.CurrentUserId] = who
                
                return await self.format_answer("user name recoded", who, sentence)
            else:
                return await self.format_answer("user shall identify first", None, sentence)
        else:
            return await self.process_question(sentence)
            
        
    async def find_first_element(self, subject:SubjectsInMemoryStore, question:str):
        """
        Find who we are speaking aabout

        Args:
            question (str): question
        """
        async def is_target(subject, question, key, value)->bool:
            """
            Decide if the suject that is refered to by the question is the one described by key:value.
            
            If it is clear that we speak about this one, return True.
            Otherwise return False.
            """
            return await emulate()
        
        for k,v in self.MemoryElements.items():
            print(f"Look: {k}, {v}")
            if await is_target(subject, question, k, v):
                print("FOUND: ", k, v)
                return k, v
            
        return None, None
            
    async def format_answer(self, instruction, data, question)->str:
        """
        Format a written answer to the question knowing that have executed `instruction` on 'data'.

        Args:
            instruction (_type_): what we have done or that we want to tell the user that we have done
            data (_type_): the data found, inserted or modified
            question (_type_): wht the used originally asked for.

        Returns:
            str: what we say to the user as an answer to his question
        """
        return await emulate()
        
        
    async def process_question(self, question):
        """
        This is the main logic for thiw workflow agent

        Args:
            question (str): user question
        """
        subject, action  = await asyncio.gather(
            self.is_about_element_that_we_record(question),
            self.detect_action_type(question)
        )
        print(subject, action)

        if subject is SubjectsInMemoryStore.NotValidSubject:
            return await self.format_answer("this suject is not handeled by this assistant", action, question)

        if action is ActionType.INSERT:
            if subject is SubjectsInMemoryStore.PeopleNames:      
                who_name, details = await asyncio.gather(
                    closure("return a name for the person that we shall remember")(question),
                    self.fill_user_details(question, None)
                )                
                data = self.MemoryElements.get(SubjectsInMemoryStore.PeopleNames, {})[who_name] = details
                return await self.format_answer("we have recorded the people", data, question )
                
            else:
                what_name, what_desc = await asyncio.gather(
                    closure("return a name for the element that we shall remember")(question),
                    closure("return a description of the element that we shall remember")(question)
                )
                if subject not in self.MemoryElements:
                    self.MemoryElements[subject] = {}
                self.MemoryElements.get(subject)[what_name] = what_desc
                return await self.format_answer("we have recorded the element", what_desc, question )
                
        elif action is ActionType.SELECT:
            key, data = await self.find_first_element(subject, question)
            
            if key is None:
                return await self.format_answer("we have not found the element in out knowledge", None, question )
                
            print("FOUND: ", key, data)
            return await self.format_answer("we have found the element in out knowledge", {"key":key, "data":data}, question)
            
        else:
            return await self.format_answer("action not yet supported", action, question)
            
        


In [83]:
A=MyAgentWithMemory()
config.DefaultPipeline.emulate_meta_prompt.source += "/no_think"

In [84]:
await A.is_telling_who_he_is("bonjour, je suis emmanuel")

True

In [85]:
await A.answer_to("Quelle sera la météo demain ?")

False


'The user shall identify first. Please provide the necessary information to answer the question about the weather tomorrow.'

In [86]:
await A.answer_to("Qui es-tu ?")

False


'Je suis un agent de suivi de la mémoire.'

In [87]:
await A.answer_to("Qui qui-je ?")

True


"Je me nomme [Nom de l'utilisateur]."

In [88]:
await A.answer_to("Je suis emmanuel batt. mémorise cela ")

SubjectsInMemoryStore.FamilyMemberLinks ActionType.INSERT
Type detection failed for return a name for the element that we shall remember: 'None' is not a valid ArgType. Returning type 'Any'.
Type detection failed for return a description of the element that we shall remember: 'None' is not a valid ArgType. Returning type 'Any'.


'what we say to the user as an answer to his question'

In [89]:
A.MemoryElements

{<SubjectsInMemoryStore.PeopleNames: 'PeopleNames'>: {'you': None},
 <SubjectsInMemoryStore.Grocery: 'Grocery'>: {'pain': 'Buy bread',
  '"ElementName"': '"Remembering the element: \'Il faut acheter du pain\'"'},
 <SubjectsInMemoryStore.FamilyMemberLinks: 'FamilyMemberLinks'>: {"'Je suis emmanuel batt. mémorise cela '": '"Je suis emmanuel batt. mémorise cela "'}}

In [90]:
await A.answer_to("Il faut acheter du pain")

SubjectsInMemoryStore.Grocery ActionType.INSERT
Type detection failed for return a name for the element that we shall remember: 'None' is not a valid ArgType. Returning type 'Any'.
Type detection failed for return a description of the element that we shall remember: 'None' is not a valid ArgType. Returning type 'Any'.


'what we say to the user as an answer to his question'

In [91]:
await A.answer_to("Que faut il acheter ?")

SubjectsInMemoryStore.Grocery ActionType.SELECT
Look: SubjectsInMemoryStore.PeopleNames, {'you': None}
Look: SubjectsInMemoryStore.Grocery, {'pain': 'Buy bread', '"ElementName"': '"Remembering the element: \'Il faut acheter du pain\'"', '"element_name"': '"we shall remember the element \'Il faut acheter du pain\'"'}
Look: SubjectsInMemoryStore.FamilyMemberLinks, {"'Je suis emmanuel batt. mémorise cela '": '"Je suis emmanuel batt. mémorise cela "'}


TypeError: cannot unpack non-iterable NoneType object

In [92]:
await A.answer_to("Faut il acheter de l'eau ?")

SubjectsInMemoryStore.Grocery ActionType.SELECT
Look: SubjectsInMemoryStore.PeopleNames, {'you': None}
Look: SubjectsInMemoryStore.Grocery, {'pain': 'Buy bread', '"ElementName"': '"Remembering the element: \'Il faut acheter du pain\'"', '"element_name"': '"we shall remember the element \'Il faut acheter du pain\'"'}
Look: SubjectsInMemoryStore.FamilyMemberLinks, {"'Je suis emmanuel batt. mémorise cela '": '"Je suis emmanuel batt. mémorise cela "'}


TypeError: cannot unpack non-iterable NoneType object

In [74]:
from OpenHosta import print_last_prompt
print_last_prompt(A.format_answer)

System prompt:
-----------------
You will act as a simulator for functions that cannot be implemented in actual code.

I'll provide you with function definitions described in Python syntax. 
These functions will have no body and may even be impossible to implement in real code, 
so do not attempt to generate the implementation.

Instead, imagine a realistic or reasonable output that matches the function description.
I'll ask questions by directly writing out function calls as one would call them in Python.
Respond with an appropriate return value, without adding any extra comments or explanations.
If the provided information isn't enough to determine a clear answer, respond simply with "None".
If assumptions need to be made, ensure they stay realistic, align with the provided description.

Here's the function definition:

```python
<class 'str'>

def format_answer(self, instruction, data, question) -> str:
    """
        Format a written answer to the question knowing that have execut